In [18]:
############# NEW BOOK TO SCRAPE USING THE NEW STRATEGY #############

###### GET ALL RESULTS FOR A GIVEN SPORT AND YEAR WITH JUST ONE QUERY 

In [19]:
## Dependencies and Setup

import pandas as pd
import numpy as np
import requests
import time
import json
import datetime
import os
import re



In [20]:
### URL BLOCK ###

# example url for softball

eaxmple_url = 'https://my.mhsaa.com/DesktopModules/MHSAA-Endpoint/handlers/ContestWithParticipantDepotSearch.ashx?Method=ScoreCenterSearch&StartDate=2023-01-26&EndDate=2023-06-26&Sport=GSB&Level=V'


## static peices of the url

url_a = 'https://my.mhsaa.com/DesktopModules/MHSAA-Endpoint/handlers/ContestWithParticipantDepotSearch.ashx?Method=ScoreCenterSearch&StartDate='
url_b = '&EndDate='
url_c = '&Sport='
url_d = '&Level=V'


### Date format ###
# YYYY-MM-DD

## academic year start and end dates
# Start August 1st
# End July 31st

# example date
# 2023-01-26


### Sport codes ###

# Ice Hockey (Boys) = BIH
# Football = BFB
# Baseball = BSB
# Softball = GSB

# Boys Basket Ball = BBB
# Girls Basket Ball = GBB

# Lacrosse (Boys) = BBL
# Lacrosse (Girls) = GGL
# Soccer (Boys) = BSO
# Soccer (Girls) = GSO


######## Sports with different score structures ########
# Volleyball = GVB
# Wrestling, Tennis (Boys and Girls), Golf (Boys and Girls), Track, Swimming, Skiiing, Cross Country








In [21]:
## Try to construct the url for a given sport and a grab 5 years worth of data

# sport code
sport_code = 'GBB'

# start date
start_date = '2022-08-01'

# end date
end_date = '2023-07-31'

# construct the url
url = url_a + start_date + url_b + end_date + url_c + sport_code + url_d

# print the url
print(url)


## Now that we have the url, we can use the requests library to get the data

# import the requests library
import requests
import json

# get the data
r = requests.get(url)

# data will be a json. display it pretty
data = r.json()
# print(json.dumps(data, indent=4, sort_keys=True))

# Output prettyfied json to text file
with open('../TEMP/data_GBB.txt', 'w') as outfile:
    json.dump(data, outfile, indent=4, sort_keys=True)


# create a dataframe from the json
df = pd.DataFrame(data)


# unique Tournament names
# df['TournamentName'].unique()





https://my.mhsaa.com/DesktopModules/MHSAA-Endpoint/handlers/ContestWithParticipantDepotSearch.ashx?Method=ScoreCenterSearch&StartDate=2022-08-01&EndDate=2023-07-31&Sport=GBB&Level=V


In [22]:
###### SIMPLE FUNCTIONS FROM THE PREVIOUS NOTEBOOKS ######

# construct the url for a given year
def url_construct(year):
    # Create start and end dates in teh YYYY-MM-DD format
    # Request one academic year of data at a time August 1st to July 31st
    start_date = str(year) + '-08-01'
    end_date = str(year + 1) + '-07-31'
    url = url_a + start_date + url_b + end_date + url_c + sport_code + url_d
    return url

# extrtact academic year from date
def get_year(date):
    return date[:4]


# If tournament name contains Regional, then regional; if quarterfinal, then quarterfinal, ect
def get_postseason_level(tournament_name):
    if tournament_name is not None:
        if "District" in tournament_name:
            return 'District'
        elif 'Regional' in tournament_name:
            return 'Regional'
        elif 'Quarter Final' in tournament_name:
            return 'Quarterfinal'
        elif 'Semifinal' in tournament_name:
            return 'Semifinal'
        elif 'Final' in tournament_name:
            return 'Final'
    # if not postseason, return none
    return None

In [23]:
############ START TO PROCESS THE DATA ############
######## SHAPES DF BY DROPPING AND RENAMING COLUMNS ########
#######  ADDS COULMN FOR SPORT NAME, YEAR, OT INFO AND FORIET INFO ########

#################### TEMP ####################
# line to add SeasonYear column to test data (will be added by a different function in final code)
df['SeasonYear'] = 2023
############################################

import pandas as pd


def preprocess(df):

    # Creating ScoreNotes column
    df['ScoreNotes'] = None
    df.loc[df['Score'].str.contains('Forfeit'), 'ScoreNotes'] = 'Forfeit'
    df.loc[df['Score'].str.contains('Forfeit'), ['HostScore', 'AwayScore']] = 0

    # Creating and cleaning up OT column
    df['OT'] = df['Score'].str.extract(r'(\(.*\))') # extract anything surrounded by parentheses
    df['OT'] = df['OT'].str.replace('(', '')
    df['OT'] = df['OT'].str.replace(')', '')

    # Dropping the Score column
    df = df.drop(columns=['Score'])

    # Assuming that the rest of the columns are already correctly named, if not, you can rename them as follows
    df.rename(columns={
        'Season_Year': 'SeasonYear',
        'SportGenderDesc': 'SportName',
        'StartDate': 'StartDate',
        'IsPostSeason': 'IsPostSeason',
        'Notes': 'Notes',
        'HostSchoolId': 'HostSchoolId',
        'HostScore': 'HostScore',
        'HostTeamName': 'HostTeamName',
        'AwaySchoolId': 'AwaySchoolId',
        'AwayScore': 'AwayScore',
        'AwayTeamName': 'AwayTeamName',
        'ContestName': 'ContestNotes',
        'TournamentName': 'TournamentName',
        'TournamentNumber': 'TournamentNumber',
    }, inplace=True)

    df['PostSeasonLevel'] = df['TournamentName'].apply(get_postseason_level)
            # add sport code column
    

    df_clean = df[['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes', 
            'OT', 'ScoreNotes',
            'HostSchoolId', 'HostScore', 'HostTeamName', 
            'AwaySchoolId', 'AwayScore', 'AwayTeamName', 
            'ContestNotes', 'TournamentNumber', 
            'PostSeasonLevel']]
    return df_clean


In [24]:
## Send the data to the preprocess function
df_clean = preprocess(df)
## Add 'GBB' as SportCode
df_clean['SportCode'] = 'GBB'


# df_clean.describe()
# df_clean.info()

C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\1460760457.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['OT'] = df['OT'].str.replace('(', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\1460760457.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['OT'] = df['OT'].str.replace(')', '')
C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\2106652538.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Spor

In [25]:




# ## Get interesting sample showing rows that have OT, Forfeit, and PostseasonLevel
# df_clean.loc[(df_clean['OT'].notnull()) | (df_clean['ScoreNotes'].notnull()) | (df_clean['PostSeasonLevel'].notnull())]

# # Sample of 20 rows from results above
# df_clean.loc[(df_clean['OT'].notnull()) | (df_clean['ScoreNotes'].notnull()) | (df_clean['PostSeasonLevel'].notnull())].sample(20)






# df_clean.columns


In [26]:
import numpy as np

################################## RENAME BACK TO DF ########################################
df = df_clean


# drop any forfieted games (score = 0)
df = df.loc[(df['HostScore'] != 0) & (df['AwayScore'] != 0)]

#### Drop null scores
df = df.dropna(subset=['HostScore', 'AwayScore'])
## scores to int
df['HostScore'] = df['HostScore'].astype(int)
df['AwayScore'] = df['AwayScore'].astype(int)


# Create new columns holding the result for the home and away teams
df['HostResult'] = np.where(df['HostScore'] > df['AwayScore'], 'W', 
                            np.where(df['HostScore'] < df['AwayScore'], 'L', 
                                     np.where(df['HostScore'] == df['AwayScore'], 'T', None)))

df['AwayResult'] = np.where(df['HostScore'] > df['AwayScore'], 'L', 
                            np.where(df['HostScore'] < df['AwayScore'], 'W', 
                                     np.where(df['HostScore'] == df['AwayScore'], 'T', None)))




In [27]:
#######  DICTIONARY OF WHAT IS A CLOSE AND BLOWOUT GAME FOR EACH SPORT ############################

### THREASHOLDS ARE BASED ON MY OWN OPIION

###
### Sport codes ###

# Ice Hockey (Boys) = BIH
# Football = BFB
# Baseball = BSB
# Softball = GSB

# Boys Basket Ball = BBB
# Girls Basket Ball = GBB

# Lacrosse (Boys) = BBL
# Lacrosse (Girls) = GGL
# Soccer (Boys) = BSO
# Soccer (Girls) = GSO
################################################################################






score_threasolds = {
    'BIH': {"close": 1, "blowout": 5},
    'BFB': {"close": 7, "blowout": 21},
    'BSB': {"close": 3, "blowout": 8},
    'GSB': {"close": 3, "blowout": 8},
    'BBB': {"close": 3, "blowout": 15},
    'GBB': {"close": 3, "blowout": 15},
    'BBL': {"close": 2, "blowout": 8},
    'GGL': {"close": 2, "blowout": 8},
    'BSO': {"close": 1, "blowout": 5},
    'GSO': {"close": 1, "blowout": 5},
}



In [28]:
## Define close and blowout games
def get_game_situation(row):
    sport = row['SportCode']  # Assuming the sport name is in the format 'BSO - Baseball'
    score_diff = abs(row['HostScore'] - row['AwayScore'])

    # Check if either score is None or NaN. If so, return 'Unknown'
    if pd.isnull(row['HostScore']) or pd.isnull(row['AwayScore']):
        return 'Unknown'
    
    if score_diff <= score_threasolds[sport]['close']:
        return 'Close'
    elif score_diff >= score_threasolds[sport]['blowout']:
        return 'Blowout'
    else:
        return 'Normal'






## Create a new dataframe column for game situation
df['GameSituation'] = df.apply(get_game_situation, axis=1)

# Seperate close and blowout games
close_games = df[df['GameSituation'] == 'Close']
blowout_games = df[df['GameSituation'] == 'Blowout']


## Define the Slices for Game Situation Catagories

# Overall (all regular and post season games)
overall = df

# Regular Season
regular_season = df[df['IsPostSeason'] == False]

# Post Season
post_season = df[df['IsPostSeason'] == True]
# If the scores are 0 drop the row
post_season = post_season.drop(post_season[(post_season['HostScore'] == 0) & (post_season['AwayScore'] == 0)].index)

# Overtime Games
overtime_games = df[df['OT'].notnull()]

# Assuming 'PostSeasonLevel' is a column with values like 'District', 'Regional', etc.
district_games = df[df['PostSeasonLevel'] == 'District']
regional_games = df[df['PostSeasonLevel'] == 'Regional']
quarterfinal_games = df[df['PostSeasonLevel'] == 'Quarterfinal']
semifinal_games = df[df['PostSeasonLevel'] == 'Semifinal']
final_games = df[df['PostSeasonLevel'] == 'Final']



In [29]:
df.sample(15)

# value counts of HostResult
df['HostResult'].value_counts()

# value counts of AwayResult

df['AwayResult'].value_counts()

L    4077
W    3616
Name: AwayResult, dtype: int64

In [30]:
### Create new dataframes organized by team of all home games and all away games
df_home = regular_season[['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes',
         'OT', 'ScoreNotes', 'HostSchoolId', 'HostScore', 'HostTeamName',
            'ContestNotes', 'TournamentNumber', 'PostSeasonLevel', 'SportCode',
            'AwaySchoolId', 'AwayScore', 'AwayTeamName', 'HostResult']]
## Assign 'SeasonType' as 'Home'
df_home['SeasonType'] = 'Home'

df_away = regular_season[['SeasonYear', 'SportName', 'StartDate', 'IsPostSeason', 'Notes',
            'OT', 'ScoreNotes', 'HostSchoolId', 'HostScore', 'HostTeamName',
            'ContestNotes', 'TournamentNumber', 'PostSeasonLevel', 'SportCode',
            'AwaySchoolId', 'AwayScore', 'AwayTeamName', 'AwayResult']]
## Assign 'SeasonType' as 'Away'
df_away['SeasonType'] = 'Away'


# df_home.rename(columns={'HostSchoolId': 'SchoolId', 'HostScore': 'Score', 'HostTeamName': 'TeamName', 'HostResult': 'Result',
#                         'AwaySchoolId': 'OpponentSchoolId', 'AwayScore': 'OpponentScore', 'AwayTeamName': 'OpponentTeamName'}, inplace=True)
# df_away.rename(columns={'AwaySchoolId': 'SchoolId', 'AwayScore': 'Score', 'AwayTeamName': 'TeamName', 'AwayResult': 'Result',
#                         'HostSchoolId': 'OpponentSchoolId', 'HostScore': 'OpponentScore', 'HostTeamName': 'OpponentTeamName'}, inplace=True)


C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\124611791.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_home['SeasonType'] = 'Home'
C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\124611791.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_away['SeasonType'] = 'Away'


In [31]:
# df_home.tail(35)

# df_away.tail(35)

In [32]:
# List of all the dataframes and their names
dataframes = [(df, 'Overall'),
              (regular_season, 'Regular Season'),
              (post_season, 'Post Season'),
              (overtime_games, 'Overtime Games'),
              (close_games, 'Close Games'),
              (blowout_games, 'Blowout Games'),
              (df_home, 'Home Games'),
              (df_away, 'Away Games'),
              (district_games, 'District Games'),
              (regional_games, 'Regional Games'),
              (quarterfinal_games, 'Quarterfinal Games'),
              (semifinal_games, 'Semifinal Games'),
              (final_games, 'Final Games')]

# Iterate over the dataframes and give some stats (length, mean, median, std, min, max)
for df, name in dataframes:
    print(f'### {name} ###')
    print(f'Length: {len(df)}')
    print(f'Mean: {df["HostScore"].mean()} - {df["AwayScore"].mean()}')
    print(f'Median: {df["HostScore"].median()} - {df["AwayScore"].median()}')
    print(f'Std: {df["HostScore"].std()} - {df["AwayScore"].std()}')
    print(f'Min: {df["HostScore"].min()} - {df["AwayScore"].min()}')
    print(f'Max: {df["HostScore"].max()} - {df["AwayScore"].max()}')
    print('')

    



# df.sample(20)
# df.columns
### Show me


### Overall ###
Length: 7693
Mean: 41.196672299493045 - 39.23436890679839
Median: 41.0 - 39.0
Std: 15.239999895973884 - 15.079355150337435
Min: 0 - 1
Max: 107 - 102

### Regular Season ###
Length: 7027
Mean: 41.19311228120108 - 38.924292016507756
Median: 41.0 - 39.0
Std: 15.199287903724295 - 15.133920270331794
Min: 0 - 1
Max: 107 - 102

### Post Season ###
Length: 666
Mean: 41.234234234234236 - 42.506006006006004
Median: 41.0 - 43.0
Std: 15.674770661077721 - 14.091669590349175
Min: 2 - 5
Max: 90 - 84

### Overtime Games ###
Length: 117
Mean: 46.97435897435897 - 47.324786324786324
Median: 46.0 - 47.0
Std: 9.710873339095592 - 10.244282728319364
Min: 24 - 25
Max: 79 - 76

### Close Games ###
Length: 606
Mean: 40.31353135313531 - 40.43399339933993
Median: 40.0 - 40.0
Std: 9.298042897503146 - 9.33020153697751
Min: 17 - 15
Max: 86 - 83

### Blowout Games ###
Length: 4438
Mean: 41.82266786840919 - 38.559035601622355
Median: 44.0 - 38.0
Std: 18.130467992979458 - 17.895535096686775
Min: 0 - 1
M

In [33]:
## Add a column called SportCode and set them all to GBB
df['SportCode'] = 'GBB'

C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\2558569482.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SportCode'] = 'GBB'


In [34]:
### Append all the dataframes together and save to a new csv
df_all = pd.DataFrame()
for df, name in dataframes:
    df_all = df_all.append(df)

df_all.to_csv('../TEMP/GBB_Notebook_test.csv', index=False)


C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\3542003534.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df)
C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\3542003534.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df)
C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\3542003534.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df)
C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\3542003534.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df)
C:\Users\Justin\AppData\Local\Temp\ipykernel_13048\3542003534.py:4: FutureWarnin

In [ ]:
example.sample(20)

In [ ]:
awertgqwerg

In [ ]:
brsatgfa

In [ ]:
# ## Display the data
# print(data)

In [ ]:
### How to parse Ice Hockey data ###

## Keys I want to keep ##
# AwaySchoolId
# AwayScore
# AwayTeamName


# ContestName
# StartDate
# ContestType

# HostSchoolId
# HostScore
# HostTeamName

# IsPostSeason

# Notes
# PostSeasonInfo
# Score # just want to keep this for now - is a string and has OT info



In [ ]:
########### NEW DATA FORMAT 
######## PARSE A SEASON AT A TIME
## Map of Json keys to column names in the dataframe

# Season_Year is the academic year in which the season started
# SportGenderDesc = Sport_Name
# StartDate = StartDate
# IsPostSeason = IsPostSeason # True or False
# Notes = Notes

# HostSchoolId = HostSchoolId
# HostScore = HostScore
# HostTeamName = HostTeamName


# AwaySchoolId = AwaySchoolId
# AwayScore = AwayScore
# AwayTeamName = AwayTeamName

# ContestName = ContestNotes   ## Has names of regular season tournaments, showcases, special events, etc.

# TournamentName = TournamentName # Has the level of postseason play and a number, use Ice Hockey as an example of how to parse. Just interested in the level of postseason play, number is also in TournamentNumber
# TournamentNumber = number of the distric, regional, ect.




In [ ]:
#####  CREATE COLUMN FOR OT AND SCORE NOTES FROM SCORE COLUMN #####

## Extract the relevant info from the score column
# If it contains 'Forfeit' add that to the ScoreNotes column 
# and zero out and values in the HostScore and AwayScore columns
df.loc[df['Score'].str.contains('Forfeit'), 'ScoreNotes'] = 'Forfeit'
df.loc[df['Score'].str.contains('Forfeit'), 'HostScore'] = 0
df.loc[df['Score'].str.contains('Forfeit'), 'AwayScore'] = 0

df['OT'] = df['Score'].str.extract(r'(\(.*\))') # extract anything surrounded by parentheses
# clean up the column by removing the parentheses
df['OT'] = df['OT'].str.replace('(', '')
df['OT'] = df['OT'].str.replace(')', '')
# Get ride of the Score string column, we have everything we need from it
df = df.drop(columns=['Score'])

In [ ]:
#### Parse the json returned and create a dataframe ####

# import pandas
import pandas as pd

# create a dataframe from the json
df = pd.DataFrame(data)

# TournamentName value counts
# print(df['TournamentName'].value_counts())
# tournamentnumbervalue counts
# print(df['TournamentNumber'].value_counts())


## Examine the dataframe by showing value counts of important columns

# print(df['ContestName'].value_counts())
# print(df['ContestType'].value_counts())
# print(df['LevelTypeCode'].value_counts())
# print(df['SeasonTypeCode'].value_counts())
# print(df['IsPostSeason'].value_counts())
# print(df['PostSeasonInfo'].value_counts())
# print(df['Notes'].value_counts())
df['LocationInfo'].value_counts()

# print(df['AwayTeamName'].value_counts())

# print(df['HostTeamName'].value_counts())

# print(df['AwayScore'].value_counts())
# print(df['HostScore'].value_counts())




# END OF GENERIC FUNCTION I AM TRYING TO MAKE TO TAKE THE NEW FORMAT GAME LEVEL DATA

In [ ]:
########### TRANSFORM THE DATA ###########
############ ICE HOCKEY SPECIFIC ###########

## add a column with just the season year
# create a function to extract the year from the start date
def get_year(date):
    return date[:4]

# apply the function to the start date column


# create a list of columns to keep
keep_cols = ['HostSchoolId', 'HostTeamName', 'HostScore',  
             'AwaySchoolId', 'AwayTeamName', 'AwayScore', 
             'OT',
                        

             'ContestName', 'StartDate', 'SeasonYear',
             'ContestType', 
             'TournamentName', 'TournamentNumber',

             'IsPostSeason', 'PostSeasonInfo', 'Notes']


In [ ]:


## create a new dataframe with just the columns we want to keep
# df = df[keep_cols]

# take a look at it
# df.sample(20)

## Value count for postseason info and notes
# print(df['PostSeasonInfo'].value_counts())
# print(df['Notes'].value_counts())
# tournament name
print(df['TournamentName'].value_counts())

df['SeasonYear'].value_counts()




In [ ]:
### Create a new column the postseason level 
# If tournament name contains Regional, then regional; if quarterfinal, then quarterfinal, ect

def get_postseason_level(tournament_name):
    if tournament_name is not None:
        if 'Regional' in tournament_name:
            return 'Regional'
        elif 'Quarterfinal' in tournament_name:
            return 'Quarterfinal'
        elif 'Semifinal' in tournament_name:
            return 'Semifinal'
        elif 'Final' in tournament_name:
            return 'Final'
    return ''

    
# apply the function to the tournament name column
df['PostSeasonLevel'] = df['TournamentName'].apply(get_postseason_level)
